<a href="https://colab.research.google.com/github/athulyas1206/Prompt2Play/blob/main/Prompt2Play_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Fine-Tune TinyLlama-1.1B with QLoRA on Support Chat Data

In [ ]:
!pip install -q bitsandbytes accelerate datasets loralib peft transformers trl


In [ ]:
!pip install -q gradio

In [3]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
model.config.use_cache = False


In [5]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


In [ ]:
import pandas as pd
df = pd.read_csv("/content/DataScience QA.csv")
df.dropna(subset=['Question', 'Answer'], inplace=True)

data = [
    {"instruction": row['Question'], "input": "", "output": row['Answer']}
    for _, row in df.iterrows()
]

dataset = Dataset.from_list(data)

def format_instruction(example):
    return f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"

dataset = dataset.map(lambda x: {"text": format_instruction(x)})


In [ ]:
def tokenize_function(example):
    tokenized = tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)


In [9]:
training_args = TrainingArguments(
    output_dir="./tinyllama-qlora-support-bot",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_dir="./logs",
    num_train_epochs=3,
    logging_steps=10,
    save_total_limit=2,
    save_strategy="epoch",
    bf16=True,
    optim="paged_adamw_8bit"
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()


In [ ]:
model.save_pretrained("tinyllama-qlora-support-bot")
tokenizer.save_pretrained("tinyllama-qlora-support-bot")


In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

instruction = "symptoms of covid19"
prompt = f"### Instruction:\n{instruction}\n\n### Response:\n"

output = pipe(prompt, max_new_tokens=100)
print(output[0]['generated_text'])


In [ ]:
import shutil
shutil.make_archive("tinyllama-qlora-support-bot", 'zip', "tinyllama-qlora-support-bot")


In [ ]:
from google.colab import files

# Download the zipped model to your local system
files.download("tinyllama-qlora-support-bot.zip")

In [ ]:
import gradio as gr
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Load model and tokenizer
model_name = "tinyllama-qlora-support-bot"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Define the response function
def get_answer(question):
    prompt = f"### Instruction:\n{question}\n\n### Response:\n"
    output = pipe(prompt, max_new_tokens=100, do_sample=True, temperature=0.7)[0]['generated_text']
    response = output.split("### Response:\n")[-1].strip()
    return response

# Gradio interface
gr.Interface(
    fn=get_answer,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question..."),
    outputs="text",
    title="TinyLlama Support Bot",
    description="Ask any data science-related question fine-tuned on your custom QA dataset!"
).launch()
